In [1]:
import percolate
from percolate.services import PostgresService

pg = PostgresService()

In [2]:
from percolate.models.p8 import Project, Agent, ModelField, LanguageModelApi

repo = pg.repository(LanguageModelApi)
repo.register(register_entities=True)

2025-01-24 16:02:08.887 | WARNING  | percolate.services.PostgresService:execute:169 - Failing to execute query model <class 'percolate.models.p8.types.LanguageModelApi'> - verbose_errors=False - relation "LanguageModelApi" already exists

2025-01-24 16:02:08.890 | WARNING  | percolate.services.PostgresService:register:94 - The table already exists - will check for schema migration or ignore
2025-01-24 16:02:09.036 | WARNING  | percolate.services.PostgresService:execute:169 - Failing to execute query model <class 'percolate.models.p8.types.LanguageModelApi'> - verbose_errors=False - relation "p8_LanguageModelApi_embeddings" already exists

2025-01-24 16:02:09.037 | WARNING  | percolate.services.PostgresService:register:107 - The embedding-associated table already exists
2025-01-24 16:02:09.038 | DEBUG    | percolate.services.PostgresService:register:111 - Updating model fields
2025-01-24 16:02:09.113 | DEBUG    | percolate.services.PostgresService:register:114 - Adding the model agent
2